In [1]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import py_lets_be_quickly_rational
import time

0) intro
1) review > only optimal stopping
2) model > stoch correl; no resim because chance player always the same
2) geodesic+grad descent
3) dqn
4) mcts

In [2]:
# s0, v0, kappa, theta, alpha=vov, rho
modelParam = np.array([
    [1.0, 0.2**2, 4.0, 0.25**2, 0.5, -0.5],
    [1.0, 0.25**2, 3.0, 0.35**2, 0.4, -0.6],
    [1.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])

n = len(modelParam)

# constraint 2 theta vbar > alpha**2
for i in range(n):
    _, _, theta, vbar, alpha, _ = modelParam[i]
    print(2*theta*vbar-alpha**2)

0.25
0.5749999999999998
0.27


In [3]:
C = np.array([
    [1.0, 0.7, 0.3],
    [0.7, 1.0, 0.5],
    [0.3, 0.5, 1.0]
])

L = np.linalg.cholesky(C)
print(C-np.matmul(L, L.T))
    
for i in range(3):
    b=np.random.multivariate_normal(np.zeros(3),0.02**2*np.eye(3))
    a = L[i,:] + b
    L[i,:] = a / np.linalg.norm(a)
print(np.matmul(L, L.T))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]]
[[1.         0.68322101 0.28692363]
 [0.68322101 1.         0.47011926]
 [0.28692363 0.47011926 1.        ]]


In [4]:
def xi(a,b,c,d,k):
    r1 = np.cos(k*np.pi*(d-a)/(b-a))*np.exp(d) - np.cos(k*np.pi*(c-a)/(b-a))*np.exp(c)
    r2 = k*np.pi/(b-a)*np.sin(k*np.pi*(d-a)/(b-a))*np.exp(d) - k*np.pi/(b-a)*np.sin(k*np.pi*(c-a)/(b-a))*np.exp(c)
    return (r1+r2) / (1+(k*np.pi/(b-a))**2)

def vpsi(a,b,c,d,ks):
    ks = np.maximum(ks,1)
    r = (np.sin(ks*np.pi*(d-a)/(b-a)) - np.sin(ks*np.pi*(c-a)/(b-a)))*(b-a)/(ks*np.pi)
    r[0] = d-c
    return r

def vv_put_orig(a, b, ks):
    r = 2 / (b-a) * (-xi(a,b,a,0,ks)+vpsi(a,b,a,0,ks))
    r[0] = r[0] / 2.0
    return r

def phi_heston2(omega, kappa,theta, alpha, rho,V0,T):
    z=omega
    tau=T
    v=V0
    vbar=theta
    lamb=kappa
    eta=alpha
    a0=lamb-rho*eta*z*1j
    gamma=np.power(eta**2*(z**2+z*1j)+a0*a0, 0.5)
    G=(a0-gamma)/(a0+gamma)
    a1=v/eta/eta*((1-np.exp(-gamma*tau))/(1-G*np.exp(-gamma*tau)))*(a0-gamma)
    a2=lamb*vbar/eta/eta*(tau*(a0-gamma)-2*np.log((1-G*np.exp(-gamma*tau))/(1-G))) #c*vbar
    return np.exp(a1+a2)

def cumulants(kappa, theta, V0, t):
    c1 = (1-np.exp(-kappa*t))*(theta-V0)/(2*kappa)-theta*t/2
    c21 = V0 / (4*kappa**3) * (4*kappa**2*(1+(rho*alpha*t-1)*np.exp(-kappa*t))
                             +kappa*(4*rho*alpha*(np.exp(-kappa*t)-1)-2*alpha**2*t*np.exp(-kappa*t))
                             +alpha**2*(1-np.exp(-2*kappa*t)))
    c22 = theta / (8*kappa**3) * (8*kappa**3*t - 8*kappa**2*(1+rho*alpha*t+(rho*alpha*t-1)*np.exp(-kappa*t))+\
                                  2*kappa*((1+2*np.exp(-kappa*t))*alpha**2*t+ 8*(1-np.exp(-kappa*t))*rho*alpha)+\
                                  alpha**2*(np.exp(-2*kappa*t)+4*np.exp(-kappa*t)-5))
    return c1, c21+c22

In [5]:
T=1
S0, V0, kappa, theta, alpha, rho = modelParam[1,:]
c1, c2 = cumulants(kappa, theta, V0, T)
L=12
a=c1 - L*c2**0.5
b=c1 + L*c2**0.5
F=1
N=50

In [6]:
Ks=np.linspace(0.95,1.05,11)
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + 1 - Ks[i])

time: 0.14562004199979128
0.95 0.09968511605031086
0.96 0.1044186612554525
0.97 0.10928112956123624
0.98 0.11427204852082093
0.99 0.11939082087068936
1.0 0.12463672787430953
1.01 0.13000893294040944
1.02 0.13550648549460667
1.03 0.14112832508270606
1.04 0.14687328568367972
1.05 0.15274010021014287
calls
0.95 0.14968511605031098
0.96 0.14441866125545255
0.97 0.13928112956123617
0.98 0.13427204852082086
0.99 0.12939082087068932
1.0 0.12463672787430946
1.01 0.12000893294040949
1.02 0.11550648549460663
1.03 0.11112832508270598
1.04 0.1068732856836796
1.05 0.1027401002101429


In [16]:
T=2
n_reset=8
n_resetPerYear=n_reset//T
n_simDaysPerReset=365//(n_resetPerYear)
n_simDaysPerYear=n_simDaysPerReset*n_resetPerYear
nt=n_simDaysPerReset*n_reset
T_reset=[n_simDaysPerReset*q for q in range(1,n_reset)]

dt=1.0/n_simDaysPerYear
sqrtDt=dt**0.5
na=C.shape[0]
ns=100000
volCorrel=0.02

In [17]:
#simulate correl
U0 = np.linalg.cholesky(C).T
U0 = np.repeat(U0[np.newaxis, :, :], ns, axis=0)
U = np.repeat(U0[np.newaxis, :, :, :], n_reset, axis=0)
for i in range(n_reset):
    A=U0 if i==0 else U[i-1,:,:,:]
    A=A + np.random.multivariate_normal(np.zeros(na),volCorrel**2*np.eye(na),size=(ns,na))
    U[i,:,:,:]=A/np.linalg.norm(A,axis=2)[:,:,np.newaxis]
#np.matmul(U[-1,0,:,:], U[-1,0,:,:].T)

In [18]:
W=np.random.normal(size=(n_reset,ns,na,n_simDaysPerReset))
print(U.shape, W.shape)
W1=np.matmul(U,W)
print(W1.shape)

(8, 100000, 3, 3) (8, 100000, 3, 91)
(8, 100000, 3, 91)


In [19]:
rho=np.ones_like(W1)
for i in range(na):
    rho[:,:,i,:]=modelParam[i,-1]
W2 = rho*W1 + np.sqrt(1-np.square(rho)) * np.random.normal(size=(n_reset,ns,na,n_simDaysPerReset))

In [20]:
X=np.zeros((n_reset,ns,na))
V=np.zeros((n_reset,ns,na))

Xr=np.tile(np.log(modelParam[:,0]),(ns,1))
Vr=np.tile(modelParam[:,1],(ns,1))

for i in range(n_reset):
    for j in range(n_simDaysPerReset):
        sqrtV = np.sqrt(Vr)
        Xr = Xr + sqrtV * sqrtDt * W1[i,:,:,j] - 0.5 * Vr * dt
        Vr = np.abs(modelParam[:,2]*(modelParam[:,3]-Vr)*dt + modelParam[:,4]*sqrtV*sqrtDt*W2[i,:,:,j])         
    X[i,:,:] = Xr
    V[i,:,:] = Vr
S=np.exp(X)

In [15]:
np.mean(S[-1,:,:],axis=0)

array([0.99862017, 1.0049377 , 1.01891727])